# Ciencia de datos aplicada
## Creción de modelo Proyecto segunda entrega

### Integrantes

Jose F. Corzo Manrique - j.corzom

Felipe A. Gutiérrez Naranjo - fa.gutierrez

Alejandro Mantilla Redondo - a.mantillar

## Contexto

La agregación del dataset se realizó con el objetivo de predecir el monto posible a ganar según el día, fecha, hora y lugar con adiciones como un clima tormentoso o si es festivo.

Dados temas de la localización del cluster de procesamiento y el gran tamaño de los datos al pasarlos por la red, solo se tuvieron en cuenta para esta entrega datos del 2019.

Tiempo de procesamiento agregación año 2019 cluster yarn de 20 nodos: 30min

##### Columnas agrupadas
    "year" > solo 2019 usado para limpieza
    "month" > valores de 1 a 12
    "dayofweek" > día de la semana valores de 1 a 7
    "dayofmonth" > día del mes valores de 1 a 31
    "hour" > hora en formato 24h valores de 0 a 23
    "PULocationID" > ID de la zona de recogida
##### Columnas calculadas
    "total_time_min" > suma del tiempo en minutos de los viajes
    "total_trip_distance" > suma del total de la distancia de los viajes en millas
    "total_fare_amount" > suma del total de la tarifa de los viajes
    "total_tip" > cantidad de viajes con propina
    "total_toll" > cantidad de viajes con recargos
    "total_amount" > suma del total del valor de los viajes
    "total_taxis" > suma del total de viajes

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
#Librería de manejo de datos
import pandas as pd

import numpy as np

# Detección de anomálias
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Arbol de decisión y sus métricas
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestRegressor
import random

#Regresión lineal y extra
from sklearn.linear_model import LinearRegression

#Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import graphviz

#Prueba chi cuadrado
from  scipy.stats import chi2_contingency

#Pandas profiling
#from pandas_profiling import ProfileReport

#Último punto
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Cargar los datos
columns = ["year","month","dayofweek","dayofmonth","hour","PULocationID","total_time_min","total_trip_distance","total_fare_amount","total_tip","total_toll","total_amount","total_taxis"]
data=pd.read_csv('Taxis_agregado_2019.csv',names=columns)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184476 entries, 0 to 1184475
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   year                 1184476 non-null  int64  
 1   month                1184476 non-null  int64  
 2   dayofweek            1184476 non-null  int64  
 3   dayofmonth           1184476 non-null  int64  
 4   hour                 1184476 non-null  int64  
 5   PULocationID         1184476 non-null  int64  
 6   total_time_min       1184476 non-null  float64
 7   total_trip_distance  1184476 non-null  float64
 8   total_fare_amount    1184476 non-null  float64
 9   total_tip            1184476 non-null  int64  
 10  total_toll           1184476 non-null  int64  
 11  total_amount         1184476 non-null  float64
 12  total_taxis          1184476 non-null  int64  
dtypes: float64(4), int64(9)
memory usage: 117.5 MB


In [4]:
#Perfilamiento de los datos originales
ProfileReport(data, title="Perfilamiento de datos")

In [5]:
#Limpieza de los años no recolectados
data_clean = data[data.year == 2019]

In [6]:
#Limpieza de los datos que se muestran con ceros o negativos en el tiempo, la tarifa y la distancia
data_clean = data_clean[data_clean.total_time_min > 0]
data_clean = data_clean[data_clean.total_trip_distance > 0]
data_clean = data_clean[data_clean.total_fare_amount > 0]

In [7]:
#Limpieza de outliers
fig1 = sns.boxplot(data=data_clean, y="total_time_min", color="teal")
fig1

In [8]:
fig1 = sns.boxplot(data=data_clean, y="total_trip_distance", color="teal")
fig1

In [9]:
fig1 = sns.boxplot(data=data_clean, y="total_fare_amount", color="teal")
fig1

In [10]:
#Eliminar datos considerados outliers
data_clean = data_clean[data_clean.total_fare_amount <= 40000]
data_clean = data_clean[data_clean.total_trip_distance <= 20000]

In [11]:
#Se elimina año dado que para esta entrega no es relevante
data_clean=data_clean.drop(columns=['year'])

In [12]:
#Conversión de datos categóricos mes
def cat_month(mes):
    if mes==1: val="January"
    elif mes==2: val="February"
    elif mes==3: val="March"
    elif mes==4: val="April"
    elif mes==5: val="May"
    elif mes==6: val="June"
    elif mes==7: val="July"
    elif mes==8: val="August"
    elif mes==9: val="September"
    elif mes==10: val="October"
    elif mes==11: val="November"
    elif mes==12: val="December"
    return val

data_clean['month']=data_clean['month'].apply(cat_month)

In [13]:
#Conversión de datos categóricos semana
def cat_semana(day):
    if day==1: val="Monday"
    elif day==2: val="Tuesday"
    elif day==3: val="Wednesday"
    elif day==4: val="Thursday"
    elif day==5: val="Friday"
    elif day==6: val="Saturday"
    elif day==7: val="Sunday"
    return val

data_clean['dayofweek']=data_clean['dayofweek'].apply(cat_semana)

In [14]:
#Conversión de datos categóricos día del mes y la hora
data_clean['dayofmonth'] = data_clean['dayofmonth'].apply(str)
data_clean['hour'] = data_clean['hour'].apply(str)

In [15]:
###ENRIQUECIMIENTO
#nombre de la zona
zones=pd.read_csv('zone_lookup.csv')
zones

In [16]:
#días festivos
holidays=pd.read_csv('Holidays_NY.csv')
holidays['DAY'] = holidays['DAY'].apply(str)
holidays

In [17]:
#Tormentas de varios tipos: nevadas, olas de calor, lluvia, vientos fuertes
heavyweather=pd.read_csv('StormEvents_NY.csv')
heavyweather['DAY'] = heavyweather['DAY'].apply(str)
heavyweather

In [18]:
#Unir datasets de enriquecimiento
data_rich = pd.merge(data_clean, zones, how='left', left_on = 'PULocationID', right_on = 'LocationID')
data_rich = pd.merge(data_rich, holidays, how='left', left_on = ['month','dayofmonth'], right_on = ['MONTH','DAY'])
data_rich = pd.merge(data_rich, heavyweather, how='left', left_on = ['month','dayofmonth'], right_on = ['MONTH','DAY'])

In [19]:
#Rellenar datos binarios y borrar columnas duplicadas y datos nulos
data_rich=data_rich.drop(columns=['PULocationID','LocationID','DAY_x','MONTH_x','MONTH_y','DAY_y'])
data_rich['Holiday'] = data_rich['Holiday'].fillna(0).apply(bool)
data_rich['Storm_Event'] = data_rich['Storm_Event'].fillna(0).apply(bool)
data_rich=data_rich.dropna()
data_rich.info()

## Creación del modelo 1: Clasificador de valor

### Después de pruebas y análisis del dataset se deciden realizar las siguientes modificaciones para el modelo 1 de clasificación
### Objetivo: con las variables categóricas se predice si la cantidad de dinero promedio a recibir por el taxista podría ser alta (por encima del cuartil 3) media (cercano a la media) o baja (por debajo del cuartil 1).

*Pregunta futura al profesorado: Cómo incluir la frecuencia de un dataset agregado como peso a un modelo de predicción.*

In [20]:
#Crear la variable categórica objetivo
data_rich['total_dollars_avg'] = data_rich['total_amount']/data_rich['total_taxis']
data_rich.describe()

,total_time_min,total_trip_distance,total_fare_amount,total_tip,total_toll,total_amount,total_taxis,total_dollars_avg
count,1.144904e+06,1.144904e+06,1.144904e+06,1.144904e+06,1.144904e+06,1.144904e+06,1.144904e+06,1.144904e+06
mean,1.375660e+03,2.298809e+02,1.006618e+03,5.257028e+01,4.461160e+00,1.435545e+03,7.757490e+01,2.300852e+01
std,2.871820e+03,6.015027e+02,2.175019e+03,1.021169e+02,2.342036e+01,3.023111e+03,1.427836e+02,1.681602e+01
min,1.000000e+00,1.000000e-02,1.000000e-02,0.000000e+00,0.000000e+00,3.100000e-01,1.000000e+00,6.200000e-02
25%,3.400000e+01,9.420000e+00,3.800000e+01,0.000000e+00,0.000000e+00,4.424000e+01,2.000000e+00,1.435436e+01
50%,1.200000e+02,3.170000e+01,1.290850e+02,3.000000e+00,1.000000e+00,1.610300e+02,8.000000e+00,1.758583e+01
75%,1.393250e+03,2.359000e+02,1.039000e+03,5.600000e+01,2.000000e+00,1.479992e+03,8.500000e+01,2.488444e+01
max,4.604400e+04,1.324582e+04,3.835619e+04,1.175000e+03,4.840000e+02,4.791928e+04,1.425000e+03,6.809000e+02


In [21]:
#valores mediana: 23, Q1: 14.3, Q3: 25.3

data_rich['amount_level'] = np.where(data_rich['total_dollars_avg'] < 1.430000e+01, 'Low', 
                                     (np.where(data_rich['total_dollars_avg'] >= 1.430000e+01, 'Medium', data_rich['total_dollars_avg'])))
data_rich['amount_level'] = np.where(data_rich['total_dollars_avg'] >= 2.567000e+01, 'High', data_rich['amount_level'])

#Creación de una columna de peso=(velocidad promedio)*(% propinas)*(% recargos)*(monto promedio)
data_rich['weight'] = (data_rich['total_trip_distance']/data_rich['total_time_min'])*((data_rich['total_tip']+1)*(data_rich['total_toll']+1)*data_rich['total_amount'])/data_rich['total_taxis']
#Normalización del peso: División por la media
data_rich['weight'] = data_rich['weight']/data_rich['weight'].mean()

In [22]:
#Perfilamiento de los datos para el modelo
ProfileReport(data_model1, title="Perfilamiento de datos")

In [24]:
#Separación de los datos en predictoras y dependiente: Se eligen todas las variables categóricas creadas que son con las que cuenta el taxista en el día de trabajo
X = pd.get_dummies(data_rich[['month','dayofweek','dayofmonth','hour','Zone','Holiday','Storm_Event','amount_level']].drop('amount_level',axis=1))
Y = data_rich['amount_level']
#Base de prueba
X_train, X_test, y_train, y_test = train_test_split( 
          X, Y, test_size = 0.25, random_state = 200)

In [25]:
#Creación del el árbol de decisión con el criterio de entropía
model_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 200,
            max_depth = 3, min_samples_leaf = 5)

model_entropy.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5,
                       random_state=200)

In [26]:
# Función de predicción
def pred(X_test, clf):
    # Predicton on test with giniIndex
    y_pred = clf.predict(X_test)
    return y_pred

# Función para calcular las métricas
def metricas(y_test, y_pred):
    print("Matriz de confusión: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Precisión : ",
    accuracy_score(y_test,y_pred)*100)
      
    print("Reporte : ",
    classification_report(y_test, y_pred))

In [27]:
#Generación de datos de métricas
metricas(y_test,pred(X_test,model_entropy))

Matriz de confusión:  [[  4234  10630  53390]
 [    31  21331  48081]
 [    78  17295 131156]]
Precisión :  54.75428507542991
Reporte :                precision    recall  f1-score   support

        High       0.97      0.06      0.12     68254
         Low       0.43      0.31      0.36     69443
      Medium       0.56      0.88      0.69    148529

    accuracy                           0.55    286226
   macro avg       0.66      0.42      0.39    286226
weighted avg       0.63      0.55      0.47    286226



In [29]:
print('Training data shape:', X_train.shape)
print('Training label shape:', y_train.shape)
print('Testing data shape:', X_test.shape)
print('Testing label shape:', y_test.shape)

Training data shape: (858678, 336)
Training label shape: (858678,)
Testing data shape: (286226, 336)
Testing label shape: (286226,)
